# Perfilamiento del DataSet

In [1]:
import re
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

In [2]:
agua = pd.read_csv('../data/consumo-agua.csv')

In [16]:
agua.sample(2)

,Geo Point,Geo Shape,consumo_total_mixto,anio,nomgeo,consumo_prom_dom,consumo_total_dom,alcaldia,colonia,consumo_prom_mixto,consumo_total,consumo_prom,consumo_prom_no_dom,bimestre,consumo_total_no_dom,gid,indice_des
6960,"19.4885632293,-99.2031811727","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",250.13,2019,Azcapotzalco,33.803735,2805.71,AZCAPOTZALCO,PUEBLO SAN JUAN TLIHUACA,50.026,3059.50,34.376404,3.660000,2,3.66,33963,BAJO
31107,"19.3508617598,-99.2206588357","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",103.70,2019,Álvaro Obregón,47.987273,1583.58,ALVARO OBREGON,AMPLIACION LAS AGUILAS,51.850,1745.23,45.927105,19.316667,3,57.95,59152,BAJO


In [3]:
def cuenta_nulos_por_columnas(df):
    """
    Función que realiza una tabla con la cuenta de missing values por columna y obtiene la proporción que estos missing
    values representan del total.
    """
    valores_nulos = df.isnull().sum()
    porcentaje_valores_nulos = 100 * df.isnull().sum() / len(df)
    tabla_valores_nulos = pd.concat([valores_nulos, porcentaje_valores_nulos], axis=1)
    tabla_valores_nulos_ordenada = tabla_valores_nulos.rename(
        columns={0: 'Missing Values', 1: '% del Total'})
    tabla_valores_nulos_ordenada = tabla_valores_nulos_ordenada[
        tabla_valores_nulos_ordenada.iloc[:, 1] != 0].sort_values(
        '% del Total', ascending=False).round(1)
    print("El dataframe tiene " + str(df.shape[1]) + " columnas.\n"
                                                     "Hay " + str(tabla_valores_nulos_ordenada.shape[0]) +
          " columnas que tienen NA's.")
    return tabla_valores_nulos_ordenada

In [5]:
cuenta_nulos_por_columnas(agua)['Missing Values'].sum()

El dataframe tiene 17 columnas.
Hay 5 columnas que tienen NA's.


26318

## Profiling del data set completo

In [37]:
def genera_profiling_general(df):
    cuenta_de_variables = len(df.columns)
    cuenta_observaciones = len(df)
    total_celdas = cuenta_de_variables*cuenta_observaciones

    # Contamos el tipo de datos del dataset
    vars_type = df.dtypes
    vars_type = pd.DataFrame(vars_type, columns = ['tipo'])

    # Asignamos un valor para cada tipo

    ## Numéricas
    cantidad_numericas = len(vars_type.loc[vars_type["tipo"] == "int64"])
    cantidad_numericas = cantidad_numericas + len(vars_type.loc[vars_type["tipo"] == "float64"])
    #print(cantidad_numericas)

    ## Fechas
    cantidad_fecha = len(vars_type.loc[vars_type["tipo"] == "Date"])
    #print(cantidad_fecha)

    ## Categoricas
    cantidad_categoricas = len(vars_type.loc[vars_type["tipo"] == "category"])
    #print(cantidad_categoricas)

    ## Texto
    cantidad_texto = len(vars_type.loc[vars_type["tipo"] == "object"])
    #print(cantidad_texto)

    # Contamos los faltantes
    nulos_totales = cuenta_nulos_por_columnas(df)['Missing Values'].sum()
    #print(nulos_totales)

    # Obtenemos el porcentaje de datos que son faltantes
    nulos_porcentaje = ((nulos_totales/(total_celdas))*100).round(1).astype(str)+'%'
    #print(nulos_porcentaje)

    # Obtenemos el total de columnas duplicadas
    ds_duplicados = df.duplicated(subset=None, keep='first')
    ds_duplicados = pd.DataFrame(ds_duplicados,columns = ['duplicated'])
    numero_de_duplicados = len(ds_duplicados.loc[ds_duplicados["duplicated"] == True])
    #print(numero_de_duplicados)

    # Obtenemos el porcentaje de duplicados
    porcentaje_de_duplicados = str(((numero_de_duplicados/(total_celdas))*100))+'%'
    #print(porcentaje_de_duplicados)

    estadisticas = ['Total de variables','Conteo de observaciones','Total de celdas',
                        'Cantidad de variables numericas','Cantidad de variables de fecha',
                        'Cantidad de variables categóricas', 'Cantidad de variables de texto',
                        'Valores faltantes','Porcentaje de valores faltantes',
                        'Renglones duplicados', 'Porcentaje de valores duplicados']

    valores_estadisticas = [cuenta_de_variables,cuenta_observaciones,total_celdas,cantidad_numericas,
                        cantidad_fecha,cantidad_categoricas,cantidad_texto,nulos_totales,nulos_porcentaje,
                        numero_de_duplicados,porcentaje_de_duplicados]

    valores = {'Estadisticas':estadisticas,'Resultado':valores_estadisticas}

    df_perfilamiento_general = pd.DataFrame(data=valores)
    return df_perfilamiento_general

In [38]:
genera_profiling_general(agua)

El dataframe tiene 17 columnas.
Hay 5 columnas que tienen NA's.


,Estadisticas,Resultado
0,Total de variables,17
1,Conteo de observaciones,71102
2,Total de celdas,1208734
3,Cantidad de variables numericas,11
4,Cantidad de variables de fecha,0
5,Cantidad de variables categóricas,0
6,Cantidad de variables de texto,6
7,Valores faltantes,26318
8,Porcentaje de valores faltantes,2.2%
9,Renglones duplicados,0
